In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import TensorDataset,DataLoader
import os

from sklearn.model_selection import KFold
from torch import nn
from torchsummary import summary
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt


from torch.optim.lr_scheduler import StepLR
import pickle
from utils import * 

d:\programfilesssd2\miniconda\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
d:\programfilesssd2\miniconda\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
d:\programfilesssd2\miniconda\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
TARGET = "EDA"
with open('DICT_EDG300.pkl', 'rb') as file:
    loaded_data = pickle.load(file)
dataX = loaded_data[TARGET]["dataX"]
dataY = loaded_data[TARGET]["dataY"]
len(dataX)

2760

In [3]:
len(dataX[0][0])

1200

In [4]:
# 十折交叉验证
kf = KFold(n_splits=5,shuffle=True,random_state=114514)
INDEX   = []
KFSORT = 3
for train_index, valid_index in kf.split(dataX):
    INDEX.append((train_index,valid_index))

# dataLoader_train = DataLoader(dataset_train, batch_size=32, shuffle=True,num_workers=4,pin_memory=True)
# dataLoader_valid = DataLoader(dataset_valid, batch_size=32, shuffle=False,num_workers=4,pin_memory=True)

dataLoader_train = DataLoader(TensorDataset(dataX[INDEX[KFSORT][0]],dataY[INDEX[KFSORT][0]]), batch_size=1280, shuffle=True,num_workers=4,pin_memory = True,prefetch_factor=8)
dataLoader_valid = DataLoader(TensorDataset(dataX[INDEX[KFSORT][1]],dataY[INDEX[KFSORT][1]]), batch_size=1280, shuffle=False,num_workers=4,pin_memory = True,prefetch_factor=8)

print(f"载入数据，训练样本{len(dataX[train_index])}个，验证样本{len(dataY[valid_index])}个。")

载入数据，训练样本2208个，验证样本552个。


In [5]:
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.hidden_size = hidden_size
        self.attention_weights = nn.Parameter(torch.Tensor(hidden_size, 1))
        nn.init.xavier_uniform_(self.attention_weights)

    def forward(self, x):
        attention_scores = torch.matmul(x, self.attention_weights)
        attention_weights = torch.softmax(attention_scores, dim=1)
        attended_x = torch.sum(x * attention_weights, dim=1)
        return attended_x

class LSTMModel(nn.Module):
    def __init__(self, input_size=1200, hidden_size=1024, num_layers=4, num_classes=3):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, bidirectional=True)
        self.attention = Attention(hidden_size * 2)  # hidden_size * 2 due to bidirectional LSTM
        self.fc = nn.Linear(hidden_size * 2, num_classes)  # hidden_size * 2 due to bidirectional LSTM

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        attended_out = self.attention(lstm_out)
        output = self.fc(attended_out)
        return output

In [ ]:
modelName = "LSTM"
target = TARGET+"300"
model = LSTMModel()
# model.load_state_dict(torch.load('./OUTPUT/model/model_LSTM_EDA300_5(1).pth'), strict=True)
    
(lossHistory,accHistory,trainAccHis) = TRAIN_CNN(model,modelName =modelName,target=target,KFSORT=KFSORT,
                                                 dataLoader_train=dataLoader_train,dataLoader_valid=dataLoader_valid,
                                                 dataCount = len(dataX[train_index]),
                                                 epochs=2000,lr=0.0001,
                                                 scheduler_step_size=500,scheduler_gamma=0.99)




epoch1: 2.19401	 acc_t[0.385]	 lr[0.0001]	 acc_val: 0.35144，	OK ====================	
epoch2: 2.19115	 acc_t[0.385]	 lr[0.0001]	 acc_val: 0.35144，	OK ====================	
epoch3: 2.18958	 acc_t[0.385]	 lr[0.0001]	 acc_val: 0.35144，	OK ====================	
epoch4: 2.18653	 acc_t[0.385]	 lr[0.0001]	 acc_val: 0.35144，	OK ====================	
epoch5: 2.18318	 acc_t[0.385]	 lr[0.0001]	 acc_val: 0.35144，	OK ====================	
epoch6: 2.18212	 acc_t[0.385]	 lr[0.0001]	 acc_val: 0.35144，	OK ====================	
epoch7: 2.17893	 acc_t[0.385]	 lr[0.0001]	 acc_val: 0.35144，	OK ====================	
epoch8: 2.17664	 acc_t[0.385]	 lr[0.0001]	 acc_val: 0.35144，	OK ====================	
epoch9: 2.17432	 acc_t[0.385]	 lr[0.0001]	 acc_val: 0.34963，	-0.18	
epoch10: 2.17047	 acc_t[0.388]	 lr[0.0001]	 acc_val: 0.34963，	-0.18	
epoch11: 2.17183	 acc_t[0.383]	 lr[0.0001]	 acc_val: 0.35144，	OK ====================	
epoch12: 2.17041	 acc_t[0.385]	 lr[0.0001]	 acc_val: 0.35144，	OK ====================	
e

epoch109: 2.13349	 acc_t[0.416]	 lr[0.0001]	 acc_val: 0.39492，	-0.18	[0.00
epoch110: 2.14093	 acc_t[0.413]	 lr[0.0001]	 acc_val: 0.39130，	-0.54	[0.00
epoch111: 2.14257	 acc_t[0.417]	 lr[0.0001]	 acc_val: 0.39130，	-0.54	[0.00
epoch112: 2.13937	 acc_t[0.413]	 lr[0.0001]	 acc_val: 0.39130，	-0.54	[0.00
epoch113: 2.13137	 acc_t[0.415]	 lr[0.0001]	 acc_val: 0.39311，	-0.36	[0.00
epoch114: 2.13647	 acc_t[0.410]	 lr[0.0001]	 acc_val: 0.39311，	-0.36	[0.00
epoch115: 2.13362	 acc_t[0.414]	 lr[0.0001]	 acc_val: 0.39130，	-0.54	[0.00
epoch116: 2.13258	 acc_t[0.417]	 lr[0.0001]	 acc_val: 0.39311，	-0.36	[0.00
epoch117: 2.13394	 acc_t[0.416]	 lr[0.0001]	 acc_val: 0.39130，	-0.54	[0.00
epoch118: 2.12769	 acc_t[0.413]	 lr[0.0001]	 acc_val: 0.39130，	-0.54	[0.00
epoch119: 2.12657	 acc_t[0.413]	 lr[0.0001]	 acc_val: 0.39311，	-0.36	[0.00
epoch120: 2.12966	 acc_t[0.413]	 lr[0.0001]	 acc_val: 0.38949，	-0.72	[0.00
epoch121: 2.12705	 acc_t[0.414]	 lr[0.0001]	 acc_val: 0.39311，	-0.36	[0.00
epoch122: 2.13016	 acc_t[

epoch218: 2.09483	 acc_t[0.408]	 lr[0.0001]	 acc_val: 0.37318，	-3.44	[0.00
epoch219: 2.09068	 acc_t[0.413]	 lr[0.0001]	 acc_val: 0.38224，	-2.53	[0.00
epoch220: 2.09890	 acc_t[0.403]	 lr[0.0001]	 acc_val: 0.375，	-3.26	[0.00
epoch221: 2.09482	 acc_t[0.403]	 lr[0.0001]	 acc_val: 0.38405，	-2.35	[0.00
epoch222: 2.09508	 acc_t[0.413]	 lr[0.0001]	 acc_val: 0.37862，	-2.89	[0.00
epoch223: 2.09080	 acc_t[0.405]	 lr[0.0001]	 acc_val: 0.35869，	-4.89	[0.00
epoch224: 2.09636	 acc_t[0.407]	 lr[0.0001]	 acc_val: 0.37681，	-3.07	[0.00
epoch225: 2.09315	 acc_t[0.409]	 lr[0.0001]	 acc_val: 0.38768，	-1.99	[0.00
epoch226: 2.09694	 acc_t[0.408]	 lr[0.0001]	 acc_val: 0.375，	-3.26	[0.00
epoch227: 2.09274	 acc_t[0.408]	 lr[0.0001]	 acc_val: 0.37318，	-3.44	[0.00
epoch228: 2.09047	 acc_t[0.409]	 lr[0.0001]	 acc_val: 0.37137，	-3.62	[0.00
epoch229: 2.08624	 acc_t[0.414]	 lr[0.0001]	 acc_val: 0.37318，	-3.44	[0.00
epoch230: 2.09278	 acc_t[0.408]	 lr[0.0001]	 acc_val: 0.38405，	-2.35	[0.00
epoch231: 2.08920	 acc_t[0.41

In [ ]:

# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:582"

In [ ]:
# 0 43
# 60 42
epochs=500


x = range(0,epochs) # x轴坐标值113  0.00009 -> 81
plt.plot(trainAccHis,c = 'r') # 参数c为color简写，表示颜色,r为red即红色

In [ ]:
x = range(0,epochs) # x轴坐标值
plt.plot(lossHistory,c = 'r') # 参数c为color简写，表示颜色,r为red即红色

In [ ]:

x = range(0,epochs) # x轴坐标值
plt.plot(accHistory,c = 'r') # 参数c为color简写，表示颜色,r为red即红色


In [ ]:

with open("./OUTPUT/loss/loss_"+modelName+"_"+TARGET+"300_5("+str(KFSORT+1)+").pkl", 'wb') as file:
    pickle.dump(dict({"loss":lossHistory,"acc":accHistory,"trainAccHis":trainAccHis}), file)

In [ ]:
STOP
model = MODEL_CNN_EDA(input_channel=1)
model = model.to(device)

epochs = 500
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001) #0.0008

criterion = torch.nn.CrossEntropyLoss()

scheduler = StepLR(optimizer, step_size=100, gamma=0.97)

lossHistory = []
accHistory = []
trainAccHis = []



early_stopping = EarlyStopping(patience=1200,filename="CNN_"+TARGET+"300_5("+str(KFSORT+1)+").pth")

for epoch in range(0,epochs):
    model.train()
    LOSS = 0
    ACC = 0
    for batch_data, batch_targets in dataLoader_train:
        optimizer.zero_grad()
        outputs = model(batch_data.to(device))
        loss = criterion(outputs, batch_targets.to(device))
        loss.backward()
        
        LOSS += loss.detach().cpu().numpy()
        ACC += (torch.max(outputs,dim=1)[1] == batch_targets.to(device)).sum()
        
        optimizer.step()
        
    scheduler.step()
    
    lossHistory.append(LOSS)
    
    print(f"epoch{epoch+1}: {str(LOSS)[0:7]}",end="\t ")
    cal = (ACC/dataCount).cpu()
    print(f"acc_t[{str(np.array(cal))[0:5]}]",end="\t ")
    trainAccHis.append(cal)
    print("lr["+str(optimizer.param_groups[0]['lr'])+"]",end="\t ")
    
#     for p in model.parameters():
#         print(p.grad.data)
    
    model.eval()
    with torch.no_grad():
        all_outputs = []
        all_targets = []
        for batch_data, batch_targets in dataLoader_valid:
            outputs = model(batch_data.to(device))
            _, predicted = torch.max(outputs.data, 1)
            all_outputs.append(predicted.cpu().numpy())
            all_targets.append(batch_targets.cpu().numpy())

        all_outputs = np.concatenate(all_outputs)
        all_targets = np.concatenate(all_targets)
        accuracy = accuracy_score(all_targets, all_outputs)
        accHistory.append(accuracy)
#         accAverHis.append(np.mean(np.array(accHistory)))
        print(f"acc_val: {str(accuracy)[0:7]}，",end="\t")
        print('OK '+'='*20 if(np.max(np.array(accHistory)) == accuracy) else str(100*(accuracy - np.max(np.array(accHistory))))[0:5],end="\t")
    
    
    if(epoch>50):
        print(f"[{ str(np.abs(np.mean(lossHistory)-np.mean(lossHistory[0:epoch-50])))[0:4] }")
#         if(np.abs(np.mean(lossHistory)-np.mean(lossHistory[0:epoch-50]))<0.05):
#             if(np.abs(100*(accuracy - np.max(np.array(accHistory))))<5):
#                 print("已提前结束。")
#                 break

                
    else:
        print("")
                    
    early_stopping(accuracy, model)
    if early_stopping.early_stop:
        print("Early stopping")
        break